In [1]:
from pykrige.ok import OrdinaryKriging

In [2]:
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from sklearn.model_selection import KFold

In [4]:
df = pd.read_csv("../data/beijinga_scaled.csv", index_col = 0)
# df = df.rename(columns={'ts': 'Time', 'station_id': 'Station'})

stations = df['station_id'].unique()
stations.sort()

splits = 6

kfout = KFold(n_splits=splits, shuffle=True, random_state=0)
kfin = KFold(n_splits=splits - 1, shuffle=True, random_state=0)

station_split = {}

for kout, (sts_ftrain_index, sts_test_index) in enumerate(kfout.split(stations)):
    station_split[kout] = {}
    for kin, (sts_train_index, sts_val_index) in enumerate(kfin.split(sts_ftrain_index)):
        sts_test = stations[sts_test_index]
        sts_val = stations[sts_ftrain_index[sts_val_index]]
        sts_train = stations[sts_ftrain_index[sts_train_index]]
        station_split[kout][kin] = {
            'test': sts_test,
            'train': sts_val,
            'pool': sts_train,
        }

In [5]:
from polire.interpolate import Kriging

In [6]:
df.head()

,latitude,longitude,ts,station_id,PM2.5
0,0.58292,0.17557,0.000000,1001.0,156.250000
0,0.58292,0.17557,0.002747,1001.0,24.833333
0,0.58292,0.17557,0.005495,1001.0,72.583333
0,0.58292,0.17557,0.008242,1001.0,30.043478
0,0.58292,0.17557,0.010989,1001.0,23.083333


In [7]:
times = df['ts'].unique()
times.sort()

In [8]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [9]:
errors = []
for kout in range(6):
    for kin in range(5):
        
        
        train_stations = list(station_split[kout][kin]['train'])
        pool_stations = list(station_split[kout][kin]['pool'])
        test_stations = list(station_split[kout][kin]['test'])

        
        test_df = df[df['station_id'].isin(test_stations)]
        pool_df = df[df['station_id'].isin(pool_stations)]
        train_df = df[df['station_id'].isin(train_stations)]


        print(kout, kin)
        
        test_error = {time_ix:np.nan for time_ix in range(29, 358)}
        


        for time_ix in range(29, 358):
            
            
            test_df = df[df['station_id'].isin(test_stations)]
            pool_df = df[df['station_id'].isin(pool_stations)]
            train_df = df[df['station_id'].isin(train_stations)]


            if time_ix % 30 == 0: # is this consitent with others? consistent.... hai
                pool_var = {}
                
                curr_timestamps = times[time_ix:time_ix+1]
                curr_timestamps.sort()


                for time in curr_timestamps:



                    try:
                        temporal_train = train_df.groupby('ts').get_group(time)
                        temporal_pool = pool_df.groupby('ts').get_group(time)
                    except KeyError:
                        print("NOTHING")
                        continue
                    try:
                        krig = Kriging(require_variance=True)
                        X_train = np.array(temporal_train[['longitude', 'latitude']])
                        y_train = np.array(temporal_train[['PM2.5']])
                        krig.fit(X_train, y_train)
                        print ('good')
                                 
                    except Exception as e:
                        print(e)
                        continue
                    
                    
                    for station in pool_stations:
                        try:
                            pool_df = temporal_pool.groupby('station_id').get_group(station)
                        except:
                            print(station, "not present")
                            continue
                        X_pool_stn = np.array(pool_df[['longitude', 'latitude']])
                        mean = krig.predict(X_pool_stn)
                        var = krig.return_variance()
                        if station not in pool_var:
                            pool_var[station] = [var.data[0]]
                        else:
                            pool_var[station] = pool_var[station].append(var.data[0])
                for i in pool_var:
                    pool_var[i] = np.sum(pool_var[i]) / len(pool_var[i])
                    

                if len(pool_var) > 0:
                    station = max(pool_var, key=pool_var.get)
                    train_stations.append(station)
                    pool_stations.remove(station)
                else:
                    print("NO POOL VAR")

#             test_df = df[df['station_id'].isin(test_stations)]
#             pool_df = df[df['station_id'].isin(pool_stations)]
#             train_df = df[df['station_id'].isin(train_stations)]


            temporal_train_df = train_df[train_df['ts'] == times[time_ix]]
            temporal_test_df = test_df[test_df['ts']==times[time_ix]]


            X_train = np.array(temporal_train_df[['longitude', 'latitude']])
            y_train = np.array(temporal_train_df[['PM2.5']])


            X_test = np.array(temporal_test_df[['longitude', 'latitude']])
            y_test = np.array(temporal_test_df[['PM2.5']])

            if X_train.shape[0] > 1 and X_test.shape[0] > 1:
                
                try:
                    krig = Kriging()
                    krig.fit(X_train, y_train)

                    pred = krig.predict(X_test)

                    print(X_train.shape, len(train_stations), len(test_stations), len(pool_stations))

                    rmse = np.sqrt(mean_squared_error(pred, y_test))
                    test_error[time_ix] = [rmse]
                except Exception as e:
                    print(e)
            else:
                print("2 years waste")

            print(time_ix)


        errors.append(((kout, kin), pd.DataFrame(test_error).mean(axis = 1)))
#         break
#     break


0 0
(6, 2) 6 6 24
29
good
1019.0 not present
1027.0 not present
1036.0 not present
(5, 2) 7 6 23
30
(7, 2) 7 6 23
31
(7, 2) 7 6 23
32
(7, 2) 7 6 23
33
(7, 2) 7 6 23
34
(7, 2) 7 6 23
35
(7, 2) 7 6 23
36
(7, 2) 7 6 23
37
(7, 2) 7 6 23
38
(7, 2) 7 6 23
39
(7, 2) 7 6 23
40
(7, 2) 7 6 23
41
(7, 2) 7 6 23
42
(7, 2) 7 6 23
43
(7, 2) 7 6 23
44
(7, 2) 7 6 23
45
(6, 2) 7 6 23
46
(6, 2) 7 6 23
47
(6, 2) 7 6 23
48
(6, 2) 7 6 23
49
(6, 2) 7 6 23
50
(6, 2) 7 6 23
51
(6, 2) 7 6 23
52
(6, 2) 7 6 23
53
(6, 2) 7 6 23
54
(6, 2) 7 6 23
55
(6, 2) 7 6 23
56
(6, 2) 7 6 23
57
(6, 2) 7 6 23
58
(6, 2) 7 6 23
59
good
1002.0 not present
(5, 2) 8 6 22
60
(7, 2) 8 6 22
61
(7, 2) 8 6 22
62
(7, 2) 8 6 22
63
(7, 2) 8 6 22
64
(7, 2) 8 6 22
65
(8, 2) 8 6 22
66
(8, 2) 8 6 22
67
(8, 2) 8 6 22
68
(8, 2) 8 6 22
69
(8, 2) 8 6 22
70
(8, 2) 8 6 22
71
(8, 2) 8 6 22
72
(8, 2) 8 6 22
73
(8, 2) 8 6 22
74
(8, 2) 8 6 22
75
(8, 2) 8 6 22
76
(8, 2) 8 6 22
77
(8, 2) 8 6 22
78
(8, 2) 8 6 22
79
(8, 2) 8 6 22
80
(8, 2) 8 6 22
81
(8, 2) 8 

(9, 2) 9 6 21
127
(9, 2) 9 6 21
128
(9, 2) 9 6 21
129
(9, 2) 9 6 21
130
(9, 2) 9 6 21
131
(9, 2) 9 6 21
132
(9, 2) 9 6 21
133
(9, 2) 9 6 21
134
(9, 2) 9 6 21
135
(9, 2) 9 6 21
136
(9, 2) 9 6 21
137
(9, 2) 9 6 21
138
(9, 2) 9 6 21
139
(9, 2) 9 6 21
140
(9, 2) 9 6 21
141
(9, 2) 9 6 21
142
(9, 2) 9 6 21
143
(9, 2) 9 6 21
144
(9, 2) 9 6 21
145
(9, 2) 9 6 21
146
(9, 2) 9 6 21
147
(9, 2) 9 6 21
148
(9, 2) 9 6 21
149
good
(9, 2) 10 6 20
150
(10, 2) 10 6 20
151
(10, 2) 10 6 20
152
(9, 2) 10 6 20
153
(10, 2) 10 6 20
154
(9, 2) 10 6 20
155
(10, 2) 10 6 20
156
(9, 2) 10 6 20
157
(8, 2) 10 6 20
158
(9, 2) 10 6 20
159
(9, 2) 10 6 20
160
(10, 2) 10 6 20
161
(10, 2) 10 6 20
162
(10, 2) 10 6 20
163
(10, 2) 10 6 20
164
(10, 2) 10 6 20
165
(10, 2) 10 6 20
166
(10, 2) 10 6 20
167
(10, 2) 10 6 20
168
(10, 2) 10 6 20
169
(10, 2) 10 6 20
170
(10, 2) 10 6 20
171
(10, 2) 10 6 20
172
(10, 2) 10 6 20
173
(10, 2) 10 6 20
174
(10, 2) 10 6 20
175
(10, 2) 10 6 20
176
(10, 2) 10 6 20
177
(10, 2) 10 6 20
178
(10, 2) 

/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/pykrige/core.py:565: RuntimeWarning: invalid value encountered in double_scalars
  (np.amax(lags) - np.amin(lags)), np.amin(semivariance)]
/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:371: RuntimeWarning: invalid value encountered in greater_equal
  return np.all((x >= lb) & (x <= ub))
/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:371: RuntimeWarning: invalid value encountered in less_equal
  return np.all((x >= lb) & (x <= ub))


(8, 2) 8 6 22
106
(8, 2) 8 6 22
107
(8, 2) 8 6 22
108
(8, 2) 8 6 22
109
(8, 2) 8 6 22
110
(8, 2) 8 6 22
111
(8, 2) 8 6 22
112
(8, 2) 8 6 22
113
(8, 2) 8 6 22
114
(8, 2) 8 6 22
115
(8, 2) 8 6 22
116
(8, 2) 8 6 22
117
(8, 2) 8 6 22
118
(8, 2) 8 6 22
119
good
(8, 2) 9 6 21
120
(9, 2) 9 6 21
121
(9, 2) 9 6 21
122
(9, 2) 9 6 21
123
(9, 2) 9 6 21
124
(9, 2) 9 6 21
125
(9, 2) 9 6 21
126
(9, 2) 9 6 21
127
(9, 2) 9 6 21
128
(9, 2) 9 6 21
129
(9, 2) 9 6 21
130
(9, 2) 9 6 21
131
(9, 2) 9 6 21
132
(9, 2) 9 6 21
133
(9, 2) 9 6 21
134
(9, 2) 9 6 21
135
(9, 2) 9 6 21
136
(9, 2) 9 6 21
137
(9, 2) 9 6 21
138
(9, 2) 9 6 21
139
(9, 2) 9 6 21
140
(9, 2) 9 6 21
141
(9, 2) 9 6 21
142
(9, 2) 9 6 21
143
(9, 2) 9 6 21
144
(9, 2) 9 6 21
145
(9, 2) 9 6 21
146
(9, 2) 9 6 21
147
(9, 2) 9 6 21
148
(9, 2) 9 6 21
149
good
(9, 2) 10 6 20
150
(10, 2) 10 6 20
151
(10, 2) 10 6 20
152
(9, 2) 10 6 20
153
(10, 2) 10 6 20
154
(10, 2) 10 6 20
155
(10, 2) 10 6 20
156
(10, 2) 10 6 20
157
(9, 2) 10 6 20
158
(9, 2) 10 6 20
159
(9

(11, 2) 11 6 19
195
(11, 2) 11 6 19
196
(11, 2) 11 6 19
197
(11, 2) 11 6 19
198
(11, 2) 11 6 19
199
(11, 2) 11 6 19
200
(11, 2) 11 6 19
201
(11, 2) 11 6 19
202
(11, 2) 11 6 19
203
(11, 2) 11 6 19
204
(11, 2) 11 6 19
205
(11, 2) 11 6 19
206
(11, 2) 11 6 19
207
(11, 2) 11 6 19
208
(11, 2) 11 6 19
209
good
(11, 2) 12 6 18
210
(12, 2) 12 6 18
211
(12, 2) 12 6 18
212
(12, 2) 12 6 18
213
(12, 2) 12 6 18
214
(12, 2) 12 6 18
215
(12, 2) 12 6 18
216
(12, 2) 12 6 18
217
(12, 2) 12 6 18
218
(12, 2) 12 6 18
219
(12, 2) 12 6 18
220
(12, 2) 12 6 18
221
(12, 2) 12 6 18
222
(12, 2) 12 6 18
223
(12, 2) 12 6 18
224
(12, 2) 12 6 18
225
(12, 2) 12 6 18
226
(12, 2) 12 6 18
227
(11, 2) 12 6 18
228
(11, 2) 12 6 18
229
(11, 2) 12 6 18
230
(11, 2) 12 6 18
231
(11, 2) 12 6 18
232
(11, 2) 12 6 18
233
(11, 2) 12 6 18
234
(11, 2) 12 6 18
235
(11, 2) 12 6 18
236
(11, 2) 12 6 18
237
(11, 2) 12 6 18
238
(11, 2) 12 6 18
239
good
(11, 2) 13 6 17
240
(13, 2) 13 6 17
241
(13, 2) 13 6 17
242
(13, 2) 13 6 17
243
(13, 2) 13

...

(6, 2) 7 6 23
40
(7, 2) 7 6 23
41
(7, 2) 7 6 23
42
(7, 2) 7 6 23
43
(7, 2) 7 6 23
44
(7, 2) 7 6 23
45
(7, 2) 7 6 23
46
(7, 2) 7 6 23
47
(7, 2) 7 6 23
48
(7, 2) 7 6 23
49
(7, 2) 7 6 23
50
(7, 2) 7 6 23
51
(7, 2) 7 6 23
52
(7, 2) 7 6 23
53
(7, 2) 7 6 23
54
(7, 2) 7 6 23
55
(7, 2) 7 6 23
56
(7, 2) 7 6 23
57
(7, 2) 7 6 23
58
(7, 2) 7 6 23
59
good
1002.0 not present
1011.0 not present
(7, 2) 8 6 22
60
(8, 2) 8 6 22
61
(8, 2) 8 6 22
62
(8, 2) 8 6 22
63
(8, 2) 8 6 22
64
(8, 2) 8 6 22
65
(8, 2) 8 6 22
66
(8, 2) 8 6 22
67
(8, 2) 8 6 22
68
(8, 2) 8 6 22
69
(8, 2) 8 6 22
70
(8, 2) 8 6 22
71
(8, 2) 8 6 22
72
(8, 2) 8 6 22
73
(8, 2) 8 6 22
74
(8, 2) 8 6 22
75
(8, 2) 8 6 22
76
(8, 2) 8 6 22
77
(8, 2) 8 6 22
78
(7, 2) 8 6 22
79
(7, 2) 8 6 22
80
(7, 2) 8 6 22
81
(8, 2) 8 6 22
82
(8, 2) 8 6 22
83
(8, 2) 8 6 22
84
(8, 2) 8 6 22
85
(8, 2) 8 6 22
86
(8, 2) 8 6 22
87
(8, 2) 8 6 22
88
2 years waste
89
zero-size array to reduction operation maximum which has no identity
NO POOL VAR
2 years waste
90
2 years w

/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/pykrige/core.py:565: RuntimeWarning: invalid value encountered in double_scalars
  (np.amax(lags) - np.amin(lags)), np.amin(semivariance)]
/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:371: RuntimeWarning: invalid value encountered in greater_equal
  return np.all((x >= lb) & (x <= ub))
/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:371: RuntimeWarning: invalid value encountered in less_equal
  return np.all((x >= lb) & (x <= ub))


(7, 2) 8 6 22
99
(7, 2) 8 6 22
100
(7, 2) 8 6 22
101
(7, 2) 8 6 22
102
(8, 2) 8 6 22
103
(8, 2) 8 6 22
104
(8, 2) 8 6 22
105
(8, 2) 8 6 22
106
(8, 2) 8 6 22
107
(8, 2) 8 6 22
108
(7, 2) 8 6 22
109
(7, 2) 8 6 22
110
(7, 2) 8 6 22
111
(7, 2) 8 6 22
112
(7, 2) 8 6 22
113
(7, 2) 8 6 22
114
(7, 2) 8 6 22
115
(8, 2) 8 6 22
116
(8, 2) 8 6 22
117
(8, 2) 8 6 22
118
(8, 2) 8 6 22
119
good
(8, 2) 9 6 21
120
(8, 2) 9 6 21
121
(8, 2) 9 6 21
122
(9, 2) 9 6 21
123
(9, 2) 9 6 21
124
(9, 2) 9 6 21
125
(9, 2) 9 6 21
126
(9, 2) 9 6 21
127
(9, 2) 9 6 21
128
(9, 2) 9 6 21
129
(9, 2) 9 6 21
130
(9, 2) 9 6 21
131
(9, 2) 9 6 21
132
(9, 2) 9 6 21
133
(9, 2) 9 6 21
134
(9, 2) 9 6 21
135
(9, 2) 9 6 21
136
(9, 2) 9 6 21
137
(9, 2) 9 6 21
138
(9, 2) 9 6 21
139
(9, 2) 9 6 21
140
(9, 2) 9 6 21
141
(9, 2) 9 6 21
142
(9, 2) 9 6 21
143
(9, 2) 9 6 21
144
(9, 2) 9 6 21
145
(9, 2) 9 6 21
146
(9, 2) 9 6 21
147
(9, 2) 9 6 21
148
(9, 2) 9 6 21
149
good
(9, 2) 10 6 20
150
(10, 2) 10 6 20
151
(10, 2) 10 6 20
152
(9, 2) 10 6 20

191
(11, 2) 11 6 19
192
(11, 2) 11 6 19
193
(11, 2) 11 6 19
194
(11, 2) 11 6 19
195
(11, 2) 11 6 19
196
(11, 2) 11 6 19
197
(11, 2) 11 6 19
198
(11, 2) 11 6 19
199
(11, 2) 11 6 19
200
(11, 2) 11 6 19
201
(11, 2) 11 6 19
202
(11, 2) 11 6 19
203
(11, 2) 11 6 19
204
(11, 2) 11 6 19
205
(11, 2) 11 6 19
206
(11, 2) 11 6 19
207
(11, 2) 11 6 19
208
(11, 2) 11 6 19
209
good
(11, 2) 12 6 18
210
(12, 2) 12 6 18
211
(12, 2) 12 6 18
212
(12, 2) 12 6 18
213
(12, 2) 12 6 18
214
(12, 2) 12 6 18
215
(12, 2) 12 6 18
216
(12, 2) 12 6 18
217
(12, 2) 12 6 18
218
(12, 2) 12 6 18
219
(12, 2) 12 6 18
220
(12, 2) 12 6 18
221
(12, 2) 12 6 18
222
(12, 2) 12 6 18
223
(12, 2) 12 6 18
224
(12, 2) 12 6 18
225
(12, 2) 12 6 18
226
(12, 2) 12 6 18
227
(12, 2) 12 6 18
228
(12, 2) 12 6 18
229
(12, 2) 12 6 18
230
(12, 2) 12 6 18
231
(12, 2) 12 6 18
232
(12, 2) 12 6 18
233
(12, 2) 12 6 18
234
(12, 2) 12 6 18
235
(12, 2) 12 6 18
236
(12, 2) 12 6 18
237
(12, 2) 12 6 18
238
(12, 2) 12 6 18
239
good
1007.0 not present
(12, 2)

264
(14, 2) 14 6 16
265
(14, 2) 14 6 16
266
(14, 2) 14 6 16
267
(14, 2) 14 6 16
268
(14, 2) 14 6 16
269
good
(14, 2) 15 6 15
270
(15, 2) 15 6 15
271
(15, 2) 15 6 15
272
(15, 2) 15 6 15
273
(15, 2) 15 6 15
274
(15, 2) 15 6 15
275
(15, 2) 15 6 15
276
(15, 2) 15 6 15
277
(15, 2) 15 6 15
278
(15, 2) 15 6 15
279
(15, 2) 15 6 15
280
(15, 2) 15 6 15
281
(15, 2) 15 6 15
282
(15, 2) 15 6 15
283
(15, 2) 15 6 15
284
(15, 2) 15 6 15
285
(15, 2) 15 6 15
286
(15, 2) 15 6 15
287
(15, 2) 15 6 15
288
(15, 2) 15 6 15
289
(15, 2) 15 6 15
290
(15, 2) 15 6 15
291
(15, 2) 15 6 15
292
(15, 2) 15 6 15
293
(15, 2) 15 6 15
294
(15, 2) 15 6 15
295
(15, 2) 15 6 15
296
(15, 2) 15 6 15
297
(15, 2) 15 6 15
298
(15, 2) 15 6 15
299
good
(15, 2) 16 6 14
300
(16, 2) 16 6 14
301
(16, 2) 16 6 14
302
(16, 2) 16 6 14
303
(16, 2) 16 6 14
304
(16, 2) 16 6 14
305
(16, 2) 16 6 14
306
(16, 2) 16 6 14
307
(16, 2) 16 6 14
308
(16, 2) 16 6 14
309
(16, 2) 16 6 14
310
(16, 2) 16 6 14
311
(16, 2) 16 6 14
312
(16, 2) 16 6 14
313
(16, 2

In [10]:
len(errors)

30

In [11]:
d = {}
for e in errors:
    temp = d.get('kout', [])
    temp.append(e[0][0])
    d['kout'] = temp
    
    temp = d.get('kin', [])
    temp.append(e[0][1])
    d['kin'] = temp
    
    temp = d.get('rmse', [])
    temp.append(e[1][0])
    d['rmse'] = temp
    

In [12]:
pd.DataFrame(d).to_csv('active_kriging_results_2.csv', index=None)

In [15]:
pd.DataFrame(d)['rmse'].mean()

16.658840526540267